# Summarise multiple `Background` runs

In [1]:
import os
import glob
import csv
import sys
import shutil
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, "./results/python")
import background as bg

In [3]:
from matplotlib import pyplot as plt

In [4]:
goodinput = ['yes', 'y', 'Y', 'Yes']
noinput = ['no', 'n', 'N', 'No']
discardinput = ['discard', 'd', 'D']
retryinput = ['retry', 'r', 'R']
manualmode = ['manual', 'm', 'M']
numaxmode = ['numax', 'n', 'N']

In [9]:
#numaxguessfiles = [
#    'projects/corot/LRc01_PSD_No_LC_Yes.csv', 
#    'projects/corot/LRc01_both.csv',
#    'projects/corot/LRc01_PSD_Yes_LC_No.csv',
#    'projects/corot/LCs_NoPSD_enero22.csv',
#]
numaxguessfiles = [
    'projects/corot/jan2022/Runs_20220124PSD/CoRoT_COR_C01R_G1.txt',
]

In [18]:
def read_initialnumax(numaxguessfiles):
    # Read in list of initial guesses of numax from manual inspection
    starlists = []

    stars = {}

    for numaxguessfile in numaxguessfiles:
        with open(numaxguessfile) as fp:
            header, *rows = csv.reader(fp)
        if 'ID' not in header:
            raise Exception(header)
        i_star = header.index('ID')
        if 'numax' in header:
            i_numax = header.index('numax')
        elif 'numax_lc' in header:
            i_numax = header.index('numax_lc')
        else:
            raise Exception(header)

        for row in rows:
            starid = row[i_star]
            starid = 'CRT%010d' % int(starid)
            stars.setdefault(starid, {"initialnumax": {}})
            assert numaxguessfile not in stars[starid]["initialnumax"]
            numax = float(row[i_numax])
            stars[starid]["initialnumax"][numaxguessfile] = numax
    return stars

In [50]:
get_finalnumax_cache = {}

In [68]:
def read_logs(stars, idstr='corot_truncated'):
    logfiles = sorted(glob.glob('./evaluation/*' + idstr + '*.csv'))
    assert logfiles
    for logfile in logfiles:
        if "_07.csv" in logfile:
            continue
        with open(logfile) as fp:
            rows = list(csv.DictReader(fp, delimiter='\t'))
        for row in rows:
            starid = row['star']
            assert starid in stars, starid
            stars[starid].setdefault('numberruns', 0)
            stars[starid]['numberruns'] += 1
            stars[starid].setdefault('notes', []).append(row['notes'])
            if row['decision'] in goodinput:
                stars[starid]['good']=True
                if starid not in get_finalnumax_cache:
                    get_finalnumax_cache[starid] = get_finalnumax(starid)
                stars[starid]['finalnumax'] = get_finalnumax_cache[starid]
            stars[starid].setdefault("decisions", []).append(row["decision"])

In [7]:
def get_finalnumax(starid):
    resultfile = max(glob.glob(
        os.path.join('./results/', starid, '*/background_parameterSummary.txt')
    ))
    sumfile = np.loadtxt(resultfile, skiprows=11)
    if sumfile.shape[0] == 6:
        # OneHarvey
        return sumfile[4][2]
    if sumfile.shape[0] == 8:
        # TwoHarvey
        return sumfile[6][2]
    raise Exception((sumfile.shape, resultfile))

In [70]:
stars = read_initialnumax(numaxguessfiles)
read_logs(stars)

In [71]:
summaryrows = []
summaryrows.append(
    [
        'star',
        'uniqueinitialnumax',
        # 'initialnumax',
        'numberruns',
        'finaleval',
        'finalnumax',
        'numaxdiff',
        'notes',
    ]
)
for i, (starid, s) in enumerate(stars.items()):
    # print(i, starid, s.get('good'), s.get('finalnumax'), *s["initialnumax"].values())
    initialnumax = repr(s["initialnumax"])
    try:
        uniqueinitialnumax_v, = set(s["initialnumax"].values())
        uniqueinitialnumax = str(uniqueinitialnumax_v)
    except ValueError:
        uniqueinitialnumax = ""
    numberruns = str(s.get("numberruns") or 0)
    finaleval = s.get("decisions", [""])[-1]
    finalnumax = str(s.get("finalnumax", ""))
    if uniqueinitialnumax and finalnumax:
        numaxdiff = str(float(finalnumax) - float(uniqueinitialnumax))
    else:
        numaxdiff = ""
    notes = "\n".join(n for n in s.get("notes", []) if n != "None")
    summaryrows.append(
        [
            starid,
            uniqueinitialnumax,
            # initialnumax,
            numberruns,
            finaleval,
            finalnumax,
            numaxdiff,
            notes,
        ]
    )
with open("Summarise.csv", "w") as fp:
    csv.writer(fp, delimiter="\t").writerows(summaryrows)
with open("Summarise.csv") as fp:
    print(fp.read())

star	uniqueinitialnumax	initialnumax	numberruns	finaleval	finalnumax	numaxdiff	notes
CRT0100430872	99.64	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 99.64, 'projects/corot/LCs_NoPSD_enero22.csv': 99.64}	1	y	104.0204775	4.380477499999998	Low signal?
CRT0100440069	53.69	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 53.69}	1	y	54.56163321	0.8716332099999988	None
CRT0100440565	28.92	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 28.92}	1	y	29.19490464	0.27490463999999903	None
CRT0100441421	35.52	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 35.52}	1	y	34.62645556	-0.8935444400000065	None
CRT0100448189	16.95	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 16.95}	1	y	17.1274819	0.17748190000000008	None
CRT0100450409	5.87	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 5.87}	2	y	6.399656566	0.5296565659999999	"None
Bad fit, numax too high"
CRT0100454011	49.29	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 49.29}	1	y	49.35106116	0.06106116000000128	None
CRT0100457878	10.56	{'projects/corot/LRc01_PSD_No_LC_Yes.csv': 10

In [49]:
get_finalnumax('CRT0100430872')

104.0204775

In [58]:
a = dict(a=1, b=2)
repr(a) == "{'a': 1, 'b': 2}"

True